In [2]:
import numpy as np
import pandas as pd
import sys
import pickle
import json

original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
from gen_feat_prost_t5 import generating_pro_feature, generating_drug_feature
sys.path = original_path

generating features using device= cuda:2


## Drug feature extracting

In [2]:
''' drugbankid2smiles '''
file_path = "dti2vec_drugbankid2smiles"
df = pd.read_csv(file_path, header=None, delimiter="\t")  # Assuming the file has no header row
print(len(df))
print(df.iloc[[0,1,300,600, 1000]])
df.columns = ["Drug_ID", "Drug"]


''' Load json file for mapping encoding & Drug ID '''

temp = []
json_file = 'node2id.json'
with open(json_file) as f:
    data = json.load(f)

for idx, item in enumerate(df["Drug_ID"].copy()) :
    for jdx, d_name in enumerate(data.keys()) :
        if d_name == item :
            temp.append(data[d_name])
            break
        elif jdx +1 == len(data):
            temp.append(np.NaN)

''' get drug features '''
# Select unique rows based on the 'Drug' column
unique_drugs_df = df.drop_duplicates(subset=['Drug']).dropna()
unique_drugs_df = generating_drug_feature(unique_drugs_df)
# print(len(unique_drugs_df))
# print(unique_drugs_df)
tmp = unique_drugs_df[['Drug_ID', 'Drug','Morgan_Features']]
drug_id_array = tmp['Drug_ID'].values
drug_array = tmp['Drug'].values
drug_feat = tmp['Morgan_Features']
drug_enco = temp

drug_feat = {'Drug_ID': drug_id_array, 'Drug': drug_array, 'Morgan_Features': drug_feat, "Drug_enco" : drug_enco}
with open("VEC_drug_feats.pkl", 'wb') as f:
   pickle.dump(drug_feat, f)

9881
            0                                                  1
0     DB00672                 CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl
1     DB00116  C1C(NC2=C(N1)N=C(NC2=O)N)CNC3=CC=C(C=C3)C(=O)N...
300   DB00419          CCCCN1C[C@@H]([C@H]([C@@H]([C@H]1CO)O)O)O
600   DB00714       CN1CCC2=C3[C@H]1CC4=C(C3=CC=C2)C(=C(C=C4)O)O
1000  DB00497  CN1CC[C@]23[C@@H]4C(=O)CC[C@]2([C@H]1CC5=C3C(=...


In [ ]:
with open('VEC_drug_feats.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x)

## Target feature extracting

### prot-bert-bfd

In [2]:
import importlib
import sys
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import gen_feat_protbert_bfd
importlib.reload(gen_feat_protbert_bfd)

sys.path = original_path
from gen_feat_protbert_bfd import generating_pro_feature, generating_drug_feature

df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "ProtBERT_Features" : new_x["ProtBERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_tmp.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")



ModuleNotFoundError: No module named 'generating_feature'

### prost_t5 -> 보류

In [2]:
import importlib
import sys
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import gen_feat_prost_t5
# import generating_feature
# importlib.reload(generating_feature)
importlib.reload(gen_feat_prost_t5)

sys.path = original_path
# from generating_feature import generating_pro_feature, generating_drug_feature
from gen_feat_prost_t5 import generating_pro_feature, generating_drug_feature


df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "ProtBERT_Features" : new_x["ProtBERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_tmp.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")



generating features using device= cuda:2
1407


Some weights of the model checkpoint at Rostlab/ProstT5 were not used when initializing T5EncoderModel: ['decoder.block.6.layer.1.EncDecAttention.q.weight', 'decoder.block.10.layer.1.EncDecAttention.v.weight', 'decoder.block.19.layer.0.SelfAttention.k.weight', 'decoder.block.21.layer.1.EncDecAttention.k.weight', 'decoder.block.2.layer.0.layer_norm.weight', 'decoder.block.8.layer.0.layer_norm.weight', 'decoder.block.6.layer.0.SelfAttention.o.weight', 'decoder.block.9.layer.1.EncDecAttention.q.weight', 'decoder.block.14.layer.0.SelfAttention.v.weight', 'decoder.block.4.layer.0.SelfAttention.v.weight', 'decoder.block.23.layer.1.EncDecAttention.k.weight', 'decoder.block.17.layer.0.SelfAttention.q.weight', 'decoder.block.20.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'decoder.block.7.layer.1.layer_norm.weight', 'decoder.block.16.layer.2.DenseReluDense.wo.weight', 'decoder.block.20.layer.1.layer_norm.weight', 'decoder.block.23.layer.0.SelfAttention.k.weigh

torch.Size([941, 2, 1024])
torch.Size([1024])
tensor([-0.0210, -0.0137, -0.0009,  ..., -0.0041, -0.0082,  0.0356],
       device='cuda:2', dtype=torch.float16)
torch.Size([389, 2, 1024])
torch.Size([1024])
tensor([-0.0209, -0.0137, -0.0009,  ..., -0.0041, -0.0081,  0.0356],
       device='cuda:2', dtype=torch.float16)
torch.Size([985, 2, 1024])
torch.Size([1024])
tensor([-0.0209, -0.0137, -0.0009,  ..., -0.0041, -0.0082,  0.0357],
       device='cuda:2', dtype=torch.float16)
torch.Size([645, 2, 1024])
torch.Size([1024])
tensor([-0.0210, -0.0137, -0.0009,  ..., -0.0041, -0.0082,  0.0356],
       device='cuda:2', dtype=torch.float16)
torch.Size([1017, 2, 1024])
torch.Size([1024])
tensor([-0.0210, -0.0137, -0.0009,  ..., -0.0041, -0.0081,  0.0356],
       device='cuda:2', dtype=torch.float16)
torch.Size([367, 2, 1024])
torch.Size([1024])
tensor([-0.0209, -0.0137, -0.0009,  ..., -0.0041, -0.0082,  0.0356],
       device='cuda:2', dtype=torch.float16)
torch.Size([3027, 2, 1024])
torch.Size(

KeyboardInterrupt: 

In [7]:
with open('VEC_target_feats_tmp.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x)

{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'ProtBERT_Features': 0       [0.08000009, 0.03540444, 0.1369079, 0.07316148...
1       [0.09861888, 0.035058178, 0.003522612, -0.0316...
2       [0.092192225, -0.029923242, 0.0971865, 0.09899...
3       [0.06595417, -0.026302155, 0.05356965, 0.03540...
4       [-0.0011727039, 0.008875381, 0.18359864, 0.049...
                              ...                        
1380    [0.08340941, 0.0069508995, 0.04182762, 0.07996...
1381    [0.10048999, -0.0183095, 0.12001386, 0.0102570...
1382    [0.053325012, -0.019652944, 0.084076114, 0.044...
1383    [-0.005918188, 0.02972834, 0.09931437, 0.00470...
1384    [0.037601013, 0.004188426, 0.10400484, -0.0012...
Name: ProtBERT_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}


### prot_t5_xl_bfd

In [2]:
import importlib
import sys
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import gen_feat_prot_t5_xl_bfd
# import generating_feature
# importlib.reload(generating_feature)
importlib.reload(gen_feat_prot_t5_xl_bfd)
sys.path = original_path
# from generating_feature import generating_pro_feature, generating_drug_feature
from gen_feat_prot_t5_xl_bfd import generating_pro_feature, generating_drug_feature


df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "ProtBERT_Features" : new_x["ProtBERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_t5_xl_bfd.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

1407


Some weights of the model checkpoint at Rostlab/prot_t5_xl_bfd were not used when initializing T5EncoderModel: ['decoder.block.9.layer.1.EncDecAttention.q.weight', 'decoder.block.20.layer.1.EncDecAttention.o.weight', 'decoder.block.23.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.0.SelfAttention.k.weight', 'decoder.block.4.layer.1.EncDecAttention.q.weight', 'decoder.block.2.layer.1.EncDecAttention.v.weight', 'decoder.block.4.layer.1.EncDecAttention.k.weight', 'decoder.block.22.layer.0.SelfAttention.k.weight', 'decoder.block.12.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.8.layer.1.EncDecAttention.k.weight', 'decoder.block.10.layer.1.EncDecAttention.v.weight', 'decoder.block.7.layer.0.SelfAttention.o.weight', 'decoder.block.20.layer.2.DenseReluDense.wo.weight', 'decoder.block.18.layer.2.DenseReluDense.wi.weight', 'decoder.block.19.layer.0.layer_norm.weight', 'decoder.block.1.layer.0.layer_norm.weight', 'decoder.block.21.

torch.Size([1, 1024, 1024])
[ 0.265864    0.2363699  -0.19544536 ... -0.08230987  0.16616802
  0.48257482]
torch.Size([1, 1024, 1024])
[ 0.17827696  0.09034206 -0.16608715 ...  0.21032272  0.7256513
  0.60169756]
torch.Size([1, 1024, 1024])
[ 0.19941539 -0.07872224 -0.02829944 ... -0.13029675  0.57204753
  0.9917669 ]
torch.Size([1, 1024, 1024])
[ 0.33217257 -0.03848412 -0.3646764  ...  0.42655393  0.51793563
  0.7115132 ]
torch.Size([1, 1024, 1024])
[-0.02119475 -0.11739744 -0.10378879 ... -0.02303022  0.65378374
  0.8964501 ]
torch.Size([1, 1024, 1024])
[ 0.2858797  -0.15516086  0.04380666 ...  0.14042492  0.45498237
  0.41104442]
torch.Size([1, 1024, 1024])
[-0.1247244  -0.06743006 -0.37314802 ...  0.2972102   0.39296004
  0.2393756 ]
torch.Size([1, 1024, 1024])
[ 0.38194957 -0.07293623 -0.10961238 ... -0.24412109  0.11763261
  0.8135064 ]
torch.Size([1, 1024, 1024])
[ 0.38001755 -0.23214196 -0.22781439 ... -0.4287119   0.4697751
  0.8886877 ]
torch.Size([1, 1024, 1024])
[ 0.3479144

In [3]:
with open('VEC_target_feats_prot_t5_xl_bfd.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x)

{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'ProtBERT_Features': 0       [0.3061483, -0.2560348, -0.41584396, 0.8509762...
1       [0.36122397, -0.1614944, -0.08925824, 0.562378...
2       [0.059911527, -0.18174909, -0.21029367, 0.7760...
3       [0.22034469, -0.024695057, -0.31216827, 0.6357...
4       [-0.15420882, -0.26272178, -0.31549728, 0.3676...
                              ...                        
1380    [0.2510656, -0.17649199, -0.20853633, 0.408869...
1381    [0.44312462, -0.22742993, 0.23731536, 0.229630...
1382    [0.20614415, -0.19024993, -0.05385681, 0.80356...
1383    [-0.19570357, -0.32050613, -0.18916883, 0.3881...
1384    [0.22904179, 0.11342351, -0.29258642, 0.893766...
Name: ProtBERT_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}


### prot-bert

In [5]:
import importlib
import sys
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import gen_feat_protbert
# import generating_feature
# importlib.reload(generating_feature)
importlib.reload(gen_feat_protbert)
sys.path = original_path
# from generating_feature import generating_pro_feature, generating_drug_feature
from gen_feat_protbert import generating_pro_feature, generating_drug_feature


df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "ProtBERT_Features" : new_x["ProtBERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_bert.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

1407


Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 1024, 1024])
tensor([-0.0185,  0.0576,  0.0214,  ...,  0.0037, -0.0264,  0.0700],
       device='cuda:2', grad_fn=<SelectBackward0>)
torch.Size([1, 1024, 1024])
tensor([-0.0389,  0.0363,  0.1766,  ..., -0.0927, -0.0029,  0.0150],
       device='cuda:2', grad_fn=<SelectBackward0>)
torch.Size([1, 1024, 1024])
tensor([ 0.0488,  0.0177,  0.0628,  ..., -0.0164, -0.0392,  0.0430],
       device='cuda:2', grad_fn=<SelectBackward0>)
torch.Size([1, 1024, 1024])
tensor([ 0.1007,  0.0336,  0.1134,  ..., -0.0599, -0.0213,  0.0171],
       device='cuda:2', grad_fn=<SelectBackward0>)
torch.Size([1, 1024, 1024])
tensor([ 0.1467,  0.0193,  0.1005,  ..., -0.0618, -0.0439,  0.0443],
       device='cuda:2', grad_fn=<SelectBackward0>)
torch.Size([1, 1024, 1024])
tensor([ 0.1546,  0.0274,  0.1077,  ..., -0.0694, -0.0355,  0.0334],
       device='cuda:2', grad_fn=<SelectBackward0>)
torch.Size([1, 1024, 1024])
tensor([ 0.1531,  0.1646,  0.1403,  ..., -0.0786,  0.0096, -0.0398],
       device='

In [3]:
with open('VEC_target_feats_prot_bert.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x)

{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'ProtBERT_Features': 0       [0.055428114, 0.028992109, 0.033637565, -0.099...
1       [0.1306466, 0.06738457, 0.08149664, -0.1383887...
2       [-0.02885853, 0.0002719852, 0.019232428, -0.13...
3       [0.14950863, 0.0025372782, 0.09173273, -0.2294...
4       [0.0337875, 0.13144645, 0.063467346, -0.226182...
                              ...                        
1380    [0.11205415, 0.019235156, 0.08165479, -0.18659...
1381    [0.024919827, 0.084282026, -0.0066118147, -0.1...
1382    [0.11519282, 0.046071086, 0.12219516, -0.20672...
1383    [0.08724613, 0.06812866, 0.065034375, -0.15398...
1384    [0.110392936, 0.0045223017, 0.07911591, -0.209...
Name: ProtBERT_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}


### prot_t5_xl_uniref50

In [1]:
import importlib
import sys
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import gen_feat_prot_t5_xl_uniref50
# import generating_feature
# importlib.reload(generating_feature)
importlib.reload(gen_feat_prot_t5_xl_uniref50)
sys.path = original_path
# from generating_feature import generating_pro_feature, generating_drug_feature
from gen_feat_prot_t5_xl_uniref50 import generating_pro_feature, generating_drug_feature


df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "ProtBERT_Features" : new_x["ProtBERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_t5_xl_uniref50.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

NameError: name 'pd' is not defined

In [5]:
with open('VEC_target_feats_prot_t5_xl_uniref50.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x)

{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'ProtBERT_Features': 0       [0.23339663, -0.16107665, -0.2483075, 0.454003...
1       [0.19799815, -0.17363514, -0.000113108945, 0.2...
2       [-0.21022151, -0.20351231, -0.03177288, 0.1806...
3       [0.16953245, -0.12390078, 0.11324732, 0.272348...
4       [0.12069299, -0.22523174, -0.17325203, 0.00566...
                              ...                        
1380    [0.1424455, -0.11220649, 0.0985885, 0.03786475...
1381    [0.16387756, -0.43449685, -0.012802006, -0.075...
1382    [0.23513082, -0.1760802, 0.14090289, 0.2739504...
1383    [0.095515124, -0.4050157, -0.016741376, 0.0647...
1384    [0.12722236, -0.07694693, 0.09211067, 0.219167...
Name: ProtBERT_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}


### prot_t5_xxl_uniref50

In [1]:
import importlib
import sys
import pandas as pd
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import gen_feat_prot_t5_xxl_uniref50
# import generating_feature
# importlib.reload(generating_feature)
importlib.reload(gen_feat_prot_t5_xxl_uniref50)
sys.path = original_path
# from generating_feature import generating_pro_feature, generating_drug_feature
from gen_feat_prot_t5_xxl_uniref50 import generating_pro_feature, generating_drug_feature


df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "ProtBERT_Features" : new_x["ProtBERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_t5_xxl_uniref50.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

1407


In [ ]:
with open('VEC_target_feats_prot_t5_xxl_uniref50.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x)